OUTLINE

MODULE 1: Data preprocessing and loading

(a) Obtaining and cleaning the datasets

(b) Twitter specific text pre-processing

(c) Word / sentence vectorisation (as first data input)

Michael: BERT and FastText

(d) Implementing a dictionary approach potentially based on Hatebase.org (as second data input)
(e) Splitting data into test, validation and testing set
(f) Specifying and implementing the data loader
(g) Testing and iterating the module
(h) Creatingmodule-specificvisualisationsforthefinal paper

MODULE 2: Model architecture and training

(a) Choosing width and depth of the model
(b) Choosingoptimizeraswellasactivationandloss functions
(c) Choosing stopping rule, regularisation, dropout, learning rates etc.
(d) Potentially performing a hyperparameter grid search
(e) Running and tracking the training
(f) Training and tuning the model based on the re- sults of the validation set
(g) Creatingmodule-specificvisualisationsforthefi- nal paper

In [12]:
import pandas as pd

In [13]:
# (a) Obtaining and cleaning the datasets
url_davidson = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/davidson%20et%20al./labeled_data.csv"
url_founta = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/data/twitter%20data/founta%20et%20al./hatespeech_text_label_vote.csv"


data_davidson = pd.read_csv(url_davidson, names=["count", "hate_speech", "offensive_language", "neither", "label", "tweet"], header=1)
data_founta = pd.read_csv(url_founta, sep='\t', names=["tweet", "label_text", "count"])

data_founta = data_founta[~data_founta.label_text.str.contains("spam")]
data_founta['label'] = data_founta.label_text.replace({'hateful': '0', 'abusive': '1', 'normal': '2'}).astype('int')
data_founta.label.hist()

# concatinating and shuffling


HTTPError: HTTP Error 404: Not Found

In [ ]:
data_davidson.label.hist()

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/mxm/Google Drive/Masterstudium/Inhalte/4th Semester/NLP/nlp-project/coding/code/exchange_base/data.csv")
df.label.hist()

In [ ]:
# trying to re-import hatebase, somehow doesn't work

'''
import json 
import requests
import pandas as pd
from hatebase import HatebaseAPI

key = open('C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/Hatebase_Key', "r").read() # insert filepath where the API key is stored
filepath = "C:/Users/Niko/Documents/NJS Cloud/1 Uni (non-Google)/0 Hertie/2 Spring 2020/NLP/full_dictionary.csv" # insert filepath where the final csv file should be stored 

hatebase = HatebaseAPI({"key": key})
filters = {"language": "eng"}
format = "json"
# initialize list for all vocabulary entry dictionaries
eng_vocab = []
response = hatebase.getVocabulary(filters=filters, format=format)
pages = response["number_of_pages"]
# fill the vocabulary list with all entries of all pages
# this might take some time...
for page in range(1, pages+1):
    filters["page"] = str(page) 
    response = hatebase.getVocabulary(filters=filters, format=format)
    eng_vocab.append(response["result"])

data = []
for r in eng_vocab:
    data.append(r["data"])
#print len(data)
listofHatewords = []

#print len(data)
for z in data:
    for a, v in z.iteritems():
        for b in v:
            listofHatewords.append(b["vocabulary"])
# print listofHatewords
listofHatewords = list(OrderedDict.fromkeys(listofHatewords))
print(len(listofHatewords))
'''


In [42]:
import csv
import sys
import nltk
import pprint
import pandas as pd
import torch

#Loading Hatebase-Data from CSV-File
'''
def csv_dict_list(variables_file):
    hatebase_dic = csv.DictReader(open("C:/Users/Niko/Documents/GitHub/nlp-project/nlp-project-1/coding/data/dictionary/hatebase/full_dictionary.csv", encoding="utf8"))
    dict_list = []
    for line in hatebase_dic:
        dict_list.append(line)
    return dict_list

listofHatewords = csv_dict_list(sys.argv[1])

hatebase_dic=[]
with open('C:/Users/Niko/Documents/GitHub/nlp-project/nlp-project-1/coding/data/dictionary/hatebase/full_dictionary.csv', 'r') as csvfile:
    csvtext = csvfile.readlines()

for row in csvtext:
    hatebase_dic.append(tuple(row.strip().split(', ')))
'''
hatebase_dic = pd.read_csv('C:/Users/Niko/Documents/GitHub/nlp-project/nlp-project-1/coding/data/dictionary/hatebase/full_dictionary.csv')

#pprint.pprint(hatebase_dic)

# Getting Twitter data
input = "https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/data.csv"
data = pd.read_csv(input)
#pprint.pprint(data)


In [47]:
# Search function

def hatesearch(data):
    frequency = 0
    hatefulness = 0
    for term in hatebase_dic:
        if term in data['tweet']:
            frequency += 1
            hatefulness = hatebase_dic['average_offensiveness']
    data['Hatefreq'] = frequency
    data['Hatefulness'] = hatefulness
    return data

data = data.apply(hatesearch, axis = 1)

#data = data.apply(lambda x : hatesearch, axis=1)

print(data)

id  label  count  \
0            0      2      4   
1            1      1      3   
2            2      1      3   
3            3      1      4   
4            4      2      5   
...        ...    ...    ...   
110743  110743      2      4   
110744  110744      2      3   
110745  110745      2      3   
110746  110746      1      4   
110747  110747      2      3   

                                                    tweet  Hatefreq  \
0       Birthday surprises on holiday in Cala Bona Mar...         0   
1       @Saunders18Haley You don't fuck with me but ju...         0   
2       Please Check Out The @bigeznurgurl . He is dop...         0   
3       i hate to sound like a cheesy old white lady b...         0   
4       @dbutters92 I always wanted one of those! I sh...         0   
...                                                   ...       ...   
110743  @FraserWhyte81 @AgentP22 @_PaulMonaghan That's...         0   
110744  o yea!!! liking the yellow:) http://t.co/2wj4wdd0  

"\ndef hatestrength(data):\n    hatefulness = 0\n    for term in hatebase_dic:\n        if term in data['tweet']:\n            hatefulness = hatebase_dic['average_offensiveness']\n    return data\n\ndata['Hatefulness'] = data.apply(lambda x : hatestrength, axis=1)\n\nprint(data)\n\n"

In [51]:
# Exporting to browse through

import json
import requests

data.to_csv('C:/Users/Niko/Documents/newdata.csv')

In [52]:
# Building a tensor out of the additional columns

import torch

HateFrequency = torch.tensor(data['Hatefreq'])
torch.save(HateFrequency, 'https://raw.githubusercontent.com/MaximilianKupi/nlp-project/master/coding/code/exchange_base/HateFreq_Tensor.pt')

ModuleNotFoundError: No module named 'torch'